In [24]:
import pandas as pd
import numpy as np

# 1. Încarcă fișierul CSV
# Asigură-te că fișierul "Date Fillout si Typeform Combinate.csv" se află în același director cu scriptul
# sau specifică calea completă către fișier.
df = pd.read_csv("Date Fillout si Typeform Combinate.csv", sep=";")

# 2. Lista exactă de coloane de simptome
# Această listă este folosită pentru a identifica coloanele pe care dorim să le normalizăm.
symptom_cols = [
    "obezitate abdominala", "rezistenta la insulina", "prediabet", "diabet zaharat tip 2",
    "sindromul ovarelor polichistice", "ficat gras", "hipertensiune arteriala",
    "dislipidemie (grăsimi crescute in sânge)", "Niciunele de mai sus",
    "slăbesc greu", "mă îngraș ușor", "depun grasime in zona abdominala",
    "oboseala permanenta", "urinare nocturna", "lipsa de energie",
    "pofte de dulce", "foame greu de controlat", "niciunele"
]

# 3. Normalizarea Coloanelor de Simptome (0/1) - Codul revizuit anterior
for col in symptom_cols:
    col_normalized_for_contains = col.lower().strip()

    if col not in df.columns:
        df[col] = 0
        continue


    def normalize_cell(cell_value, col_normalized_name):
        try:
            num_val = int(float(str(cell_value).strip()))
            if num_val == 0 or num_val == 1:
                return num_val
        except ValueError:
            pass

        text_val = str(cell_value).lower().strip()
        if col_normalized_name in text_val:
            return 1
        return 0


    df[col] = df[col].apply(lambda x: normalize_cell(x, col_normalized_for_contains))

# --- 4. Curățarea și Conversia Datelor Numerice cu Formatare Specifică și Regula Circumferinței ---

# Definirea coloanelor numerice cheie
numeric_cols = [
    "*Vârstă*",
    "*Care este înălțimea ta? *",
    "*Care este greutatea ta actuala?*",
    "*Care este circumferința taliei tale, măsurata deasupra de ombilicului?*"
]

# Iterează prin fiecare coloană numerică pentru curățare și conversie
for col in numeric_cols:
    # Curăță valorile: elimină unități de măsură (m, kg, cm) și spații
    # Folosim .astype(str) pentru a ne asigura că operăm pe șiruri
    cleaned_values = df[col].astype(str).str.replace('m', '').str.replace('kg', '').str.replace('cm', '').str.strip()

    # Converteste coloana la tip numeric (float).
    # `errors='coerce'` va transforma valorile care nu pot fi convertite în NaN (Not a Number).
    df[col] = pd.to_numeric(cleaned_values, errors='coerce')

    # Logica specifică de conversie și formatare
    if col == "*Care este înălțimea ta? *":
        # Verifică dacă valoarea este sub 10 (indica metri) și o convertește în cm
        # Această operație se aplică doar la valorile non-NaN
        df[col] = df[col].apply(lambda x: x * 100 if pd.notna(x) and x < 10 else x)
        # Corecție pentru înălțimi cu o cifră în plus (ex: 1700 în loc de 170)
        # Un om rar depășește 250 cm. Dacă valoarea e mult prea mare, împărțim la 10.
        df.loc[df[col].notna() & (df[col] > 250), col] = df.loc[df[col].notna() & (df[col] > 250), col] / 10
        df[col] = df[col].round(1)  # Rotunjește la o zecimală
    elif col == "*Care este greutatea ta actuala?*":
        # Corecție pentru greutăți cu o cifră în plus (ex: 800 în loc de 80)
        # O greutate umană rar depășește 250-300 kg. Folosim 250 kg ca prag.
        df.loc[df[col].notna() & (df[col] > 250), col] = df.loc[df[col].notna() & (df[col] > 250), col] / 10
        df[col] = df[col].round(1)  # Rotunjește la o zecimală
    elif col == "*Care este circumferința taliei tale, măsurata deasupra de ombilicului?*":
        # Corecție pentru circumferințe cu o cifră în plus (ex: 1000 în loc de 100)
        # O circumferință a taliei rar depășește 200 cm.
        df.loc[df[col].notna() & (df[col] > 200), col] = df.loc[df[col].notna() & (df[col] > 200), col] / 10
        df[col] = df[col].round(1)  # Rotunjește la o zecimală

    # Regula specifică pentru circumferința taliei (sub 45 cm)
    if col == "*Care este circumferința taliei tale, măsurata deasupra de ombilicului?*":
        # Identifică valorile care sunt mai mici de 45 și le transformă în NaN
        # Acestea vor fi apoi completate cu mediana.
        df.loc[df[col] < 45, col] = np.nan

    # Gestionează valorile lipsă (NaN) prin imputare cu mediana coloanei.
    # Mediana este o alegere bună deoarece este mai puțin afectată de valori extreme (outliers).
    # Se recalculează mediana după conversiile unitare și tratarea valorilor sub 45.
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

    # Asigură-te că vârsta este un int (nu are sens să fie float)
    if col == "*Vârstă*":
        df[col] = df[col].astype(int)

# --- 5. Preprocesarea Coloanelor Categorice (Rămâne doar pentru calculul IMC, dacă e cazul) ---

# Nu mai este necesară o coloană numerică pentru gen pentru IMC, dar o lăsăm dacă e nevoie pentru alte scopuri
# Rămâne doar curățarea, dar nu și maparea la 0/1 specific pentru IMC.
df['*Ești *'] = df['*Ești *'].astype(str).str.lower().str.strip()

# --- 6. Calculul IMC (Indice de Masă Corporală) ---
# Formula IMC: Greutate (kg) / (Înălțime (m))^2

# Asigură-te că coloanele necesare sunt numerice și în unitățile corecte
# Înălțimea a fost deja convertită în cm, o convertim în metri pentru formula IMC
height_m = df['*Care este înălțimea ta? *'] / 100
weight_kg = df['*Care este greutatea ta actuala?*']

# Calculează IMC
df['IMC'] = weight_kg / (height_m ** 2)

df['IMC'] = df['IMC'].round(1)  # Rotunjește IMC la o zecimală

# 7. Afișează primele rânduri din coloanele preprocesate pentru verificare
# Acum include și coloanele numerice preprocesate, inclusiv IMC.
print("DataFrame după normalizarea simptomelor, preprocesarea numerică și calculul IMC:")
print(df[symptom_cols + numeric_cols + ['IMC']].head())

# Afișează tipurile de date pentru a verifica dacă conversia a fost corectă
print("\nTipuri de date pentru coloanele preprocesate (inclusiv IMC):")
print(df[symptom_cols + numeric_cols + ['IMC']].dtypes)

df.head(4172)


DataFrame după normalizarea simptomelor, preprocesarea numerică și calculul IMC:
   obezitate abdominala  rezistenta la insulina  prediabet  \
0                     1                       0          0   
1                     1                       0          0   
2                     0                       0          0   
3                     1                       1          0   
4                     1                       0          0   

   diabet zaharat tip 2  sindromul ovarelor polichistice  ficat gras  \
0                     0                                0           0   
1                     0                                1           1   
2                     0                                0           1   
3                     0                                1           0   
4                     0                                0           0   

   hipertensiune arteriala  dislipidemie (grăsimi crescute in sânge)  \
0                        0               

,*Vârstă*,*Ești *,*Unde locuiești*?,*Ocupatia*,*Care este înălțimea ta? *,*Care este greutatea ta actuala?*,"*Care este circumferința taliei tale, măsurata deasupra de ombilicului?*",obezitate abdominala,rezistenta la insulina,prediabet,...,foame greu de controlat,niciunele,*Ce alte simptome sau boli prezinți?*,*Care sunt obiectivele tale legate de sănătate? Ce îți dorești sa obții? *,*Greutatea la care vrei sa ajungi?*,"*In prezent, care este cea mai mare provocare a ta? Ce crezi ca te împiedica sa slăbești si sa ai o stare buna de sănătate? *",*De ce îți dorești sa slăbești?*,*Ai mai încercat in trecut sa slăbești? *,*Ce te-a împiedicat in trecut sa slăbești? De ce ai eșuat la alte încercări? *,IMC
0,45,femeie,Londra,Angajat,166.0,82.0,100.0,1,0,0,...,1,0,Infecții urinare dese,"Energie, scăderea în greutate, tonus și o alim...",57kg,"Stresul, starea mea emoțională nu este una bun...","Pentru a mea sănătate, fizică și psihică, ca s...",1,Lipsa consecvenți.,29.8
1,38,femeie,Ilfov,Angajat,170.0,72.0,100.0,1,0,0,...,0,0,Litiaza renala,Sa scap de constipatie si grasimea abdominala,65kg,Balonarea si constipatia,Sa ma simt mai bine,1,Problemele de sanatate,24.9
2,35,femeie,București,Angajat,171.1,69.0,69.0,0,0,0,...,0,0,Na,Valori scazute in sange,61,Sedentarismul,Stare generala buna,1,Am slabit dar nu suficient,23.6
3,20,femeie,Alexandria,Angajat,160.0,72.0,100.0,1,1,0,...,0,0,Cele menționate sunt simptomele,Să slăbesc și să îmi reglez menstruatia,60,Sedentarismul și dulcele,"M am îngrășat prea tare, îmi afectează sanatatea",1,Am slăbit și după nu m am mai continuat,28.1
4,38,femeie,ORADEA,Antreprenor,160.0,60.0,80.0,1,0,0,...,0,0,"ENDOMETRIOZA, HEMOROIZI, CHISTURI SIMPLE SANI","STARE DE BINE, REDUCEREA GRASIMI ABDOMINALE",52,MOTIVATIA,SA MA SIMT MAI BINE,1,SUSTINEREA,23.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4167,38,femeie,Cluj-Napoca,Angajat,168.0,59.0,68.0,0,1,0,...,0,0,Infertilitate,Evitarea diabetului,56.0,Lipsa de cunostinte destule despre cum pot red...,Pentru sanatate,Nu,Nu am incercat,20.9
4168,27,femeie,București,Angajat,167.0,89.0,102.0,1,0,0,...,1,0,Mi au iesir trigliceridele mari 231,Sa slăbesc minim 10 Kg si sa fiu in parametrii...,70.0,"Pofta de dulce uneori, uneori ronțăit la birou...",Sa fiu mai sănătoasă si sa am o stare mai buna...,Da,Am încercat am slabit aproximativ 9 kg tot de...,31.9
4169,27,femeie,București,Angajat,167.0,89.0,102.0,1,0,0,...,1,0,Mi au iesir trigliceridele mari 231,Sa slăbesc minim 10 Kg si sa fiu in parametrii...,70.0,"Pofta de dulce uneori, uneori ronțăit la birou...",Sa fiu mai sănătoasă si sa am o stare mai buna...,Da,Am încercat am slabit aproximativ 9 kg tot de...,31.9
4170,48,femeie,Brașov,Altceva,162.0,85.0,112.0,1,1,1,...,1,0,Tiroidita autoimuna Hashimoto cu hipotiroidie ...,"Scadere insulinorezistenta, scadere in greutat...",65.0,Stilul de viata sedentar si alimentatia incorecta,Pt o stare de sanatate fizica si psihica mai buna,Da,Nu am facut din regimul dat un stil de viata i...,32.4


In [25]:
df.to_csv("Date Preprocesate Model.csv", index=False, sep=";")
print("\nDataFrame-ul preprocesat a fost salvat în 'Date Preprocesate Model.csv'.")


DataFrame-ul preprocesat a fost salvat în 'Date Preprocesate Model.csv'.
